# Creacion de los archivos necesarios para el pipeline de bell

### El primer archivo con el que se va a trabajar es con TAIR10_3_utr_20101028.fasta

Se extraen todos los encabezados del fasta de la siguiente forma:

In [55]:
%%bash
cd ../Transcritos
grep '>' TAIR10_3_utr_20101028.fasta | sed -e 's/|[^|]*|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t'>  TAIR10_3_utr_20101028_encabezados.txt
head TAIR10_3_utr_20101028_encabezados.txt
cp TAIR10_3_utr_20101028_encabezados.txt ../GTF/

AT1G51370.2	chr1:19046749-19046825	FORWARD	77
AT1G50920.1	chr1:18872571-18872930	FORWARD	260
AT1G44020.1	chr1:16716112-16716691	REVERSE	580
AT1G15970.1	chr1:5486319-5486543	REVERSE	225
AT1G73440.1	chr1:27612183-27612815	FORWARD	633
AT1G75120.1	chr1:28196857-28197021	REVERSE	165
AT1G17600.1	chr1:6052936-6053025	REVERSE	90
AT1G51380.1	chr1:19049968-19050162	FORWARD	195
AT1G77370.1	chr1:29074643-29074796	FORWARD	154
AT1G10950.1	chr1:3663623-3663984	FORWARD	362


Ahora se extraen unicamente los 3 UTR del GTF:


In [54]:
%%bash
grep -v '#!\|##' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr' > GTF_three_prime_utr.txt
head GTF_three_prime_utr.txt

1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.1"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-203"; transcript_source "araport11"; transcript_biotype "protein_coding"; tag "Ensembl_canonical";
1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.4"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-202"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10943007	10943138	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-204"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10942663	10942909	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene

Como se puede apreciar en la columna 9 vienen varios parametros, por lo que se van a separar en diferentes columnas con el siguiente comando:

In [53]:
%%bash
awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}' GTF_three_prime_utr.txt | sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|awk -F'\t' 'BEGIN{OFS="\t"} {$1 = "chr" $1; print}'> GTF_three_prime_utr_final.txt
head GTF_three_prime_utr_final.txt

chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.1	AT1G30814	AT1G30814-203
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.4	AT1G30814	AT1G30814-202
chr1	araport11	three_prime_utr	10943007	10943138	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942663	10942909	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.3	AT1G30814	AT1G30814-201
chr1	araport11	three_prime_utr	29677904	29678284	.	-	.	AT1G78930	AT1G78930.1	AT1G78930	AT1G78930-201
chr1	araport11	three_prime_utr	26966558	26966688	.	+	.	AT1G71695	AT1G71695.1	AT1G71695	AT1G71695-201
chr1	araport11	three_prime_utr	21805932	21806278	.	-	.	AT1G58983	AT1G58983.1	AT1G58983	AT1G58983-201
chr1	araport11	three_prime_utr	4430778	4430965	.	+	.	AT1G12980	AT1G12980.1	ESR1	ESR1-201
chr1	araport11	three_prime_utr	17141620	17141771	.	+	.	AT1G45223	AT1G45223.1	AT1G45223	AT1G45223-201


Recordando que la columna 9 es **gene_id**, 10 es **transcript_id**, 11 es **gene_name** y 12 **transcript_name**

Teniendo estos dos archivos  
*TAIR10_3_utr_20101028_encabezados.txt*  
&  
*GTF_three_prime_utr_final.txt*  
  
Se va a llevar la comparacion linea por linea del fasta con el archivo del GTF para generar el archivo con las siguientes caracteristicas:

<table>
  <thead>
    <tr style="background-color: Turquoise; color: black;">
      <th>COL1</th>
      <th>COL2</th>
      <th>COL3</th>
      <th>COL4</th>
      <th>COL5</th>
      <th>COL6</th>
      <th>COL7</th>
      <th>COL8</th>
      <th>COL9</th>
      <th>COL10</th>
      <th>COL11</th>
      <th>COL12</th>
    </tr>
  <thead>
    <tr style="background-color: Aquamarine; color: black;">
      <th>Trancript_id</th>
      <th>Type</th>
      <th>Localization</th>
      <th>Chromosome</th>
      <th>Start</th>
      <th>End</th>
      <th>Strand</th>
      <th>Exon</th>
      <th>Gene_name</th>
      <th>Gene_id</th>
      <th>Gene_id2</th>
      <th>Length</th>
    </tr>
    <tr style="background-color: Aquamarine; color: black;">
      <th>COL1 -> TAIR10_3_utr</th>
      <th>COL3 -> GTF_three_prime</th>
      <th>COL2 -> TAIR10_3_utr</th>
      <th>COL1 -> GTF_three_prime</th>
      <th>COL4 -> GTF_three_prime</th>
      <th>COL5 -> GTF_three_prime</th>
      <th>COL7-> GTF_three_prime</th>
      <th>COL6 -> GTF_three_prime</th>
      <th>COL11 -> GTF_three_prime</th>
      <th>COL9 -> GTF_three_prime_utr</th>
      <th>COL9 -> GTF_three_prime_utr</th>
      <th>COL4 -> TAIR10_3_utr</th>
    </tr> 
       


Teniendo el contenido de la tabla se ejecuta el siguiente comando para generar el archivo 3' UTR:

In [52]:
%%bash
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt | head

awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt | wc -l

AT1G30814.1 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10943007 10943138 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10942663 10942909 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.3 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G78930.1 three_prime_utr chr1:29678097-29678284 chr1 29677904 29678284 - . AT1G78930 AT1G78930 AT1G78930 188
AT1G71695.1 three_prime_utr chr1:26966558-26966688 chr1 26966558 26966688 + . AT1G71695 AT1G71695 AT1G71695 131
AT1G58983.1 three_prime_utr chr1:21806020-21806278 chr1 21805932 21806278 - . AT1G58983 AT1G58983 AT1G58983 259
AT1G12980.1 three_prime_utr chr1:4430778-4430965 chr1 4430778 4430965 + . ESR1 AT1G12980 AT1G12980 188
AT1G45223.1 three_prime_utr chr1:17141620-17141771 chr1 17141620 17141771 + . AT1G45223 AT1G45223 AT1G45223 152
A

Veo varios puntos negativos, el primero es que las coordenadas de ambos archivos no corresponden

In [51]:
%%bash
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt > ../coords_dict/TAIR10_3_UTR.coords.dict

Por lo que se tendra que volver a revisar varios parametros

El primero tiene que ver con la cantidad de anotaciones en el GTF y el archivo FASTA.

Para eso nos cambiamos de ubicacion y descargamos el archivo de Araport11:

In [50]:
%%bash
cd ../PRUEBAS
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_3_utr_20220914.gz"
gzip -d *gz
mv Araport11_3_utr_20220914 Araport11_3_utr_20220914.fasta

--2023-05-23 19:27:17--  https://www.arabidopsis.org/download_files/Sequences/Araport11_blastsets/Araport11_3_utr_20220914.gz
Resolving www.arabidopsis.org (www.arabidopsis.org)... 52.88.10.157
Connecting to www.arabidopsis.org (www.arabidopsis.org)|52.88.10.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3809852 (3.6M) [application/x-gzip]
Saving to: ‘Araport11_3_utr_20220914.gz’

     0K .......... .......... .......... .......... ..........  1%  363K 10s
    50K .......... .......... .......... .......... ..........  2%  583K 8s
   100K .......... .......... .......... .......... ..........  4% 41.6M 5s
   150K .......... .......... .......... .......... ..........  5%  667K 5s
   200K .......... .......... .......... .......... ..........  6% 4.74M 4s
   250K .......... .......... .......... .......... ..........  8% 40.7M 4s
   300K .......... .......... .......... .......... ..........  9% 57.7M 3s
   350K .......... .......... .......... .......... 

Contamos el numero de anotaciones que tiene este archivo:

In [49]:
%%bash
cd ../PRUEBAS
grep '>' Araport11_3_utr_20220914.fasta |awk '{$1=$1};1' OFS='\t'|sed -e 's/>//g'|cut -f1 |sort|uniq|wc -l

42347


Lo mismo para el archivo GTF

In [48]:
%%bash
 awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}' GTF_three_prime_utr.txt | sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|awk -F'\t' 'BEGIN{OFS="\t"} {$1 = "chr" $1; print}'|cut -f10|sort|uniq|wc -l

42322


Como se puede apreciar, no coinciden, se tiene 25 menos en el GTF

Y se procede a repetir los mismos pasos anteriores, pero con el nuevo archivo:

In [47]:
%%bash
cd ../PRUEBAS/
grep '>' Araport11_3_utr_20220914.fasta | sed -e 's/|[^|]*|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t'> Araport11_3_utr_20220914.txt
head Araport11_3_utr_20220914.txt

AT1G01010.1	chr1:5631-5899	FORWARD	269
AT1G01020.1	chr1:6788-6914	REVERSE	127
AT1G01020.2	chr1:6788-7314	REVERSE	440
AT1G01020.3	chr1:6788-6914	REVERSE	127
AT1G01020.4	chr1:6788-6914	REVERSE	127
AT1G01020.5	chr1:6788-6914	REVERSE	127
AT1G01020.6	chr1:6788-7314	REVERSE	440
AT1G01030.1	chr1:11649-11863	REVERSE	215
AT1G01030.2	chr1:11649-11863	REVERSE	215
AT1G01040.1	chr1:31080-31227	FORWARD	148


Aqui pongo un print sin cambiar la posicion de acuerdo al sentido de la cadena.

In [46]:
%%bash
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_encabezados.txt GTF_three_prime_utr_final.txt|head 

AT1G30814.1 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10943007 10943138 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10942663 10942909 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.3 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G78930.1 three_prime_utr chr1:29678097-29678284 chr1 29677904 29678284 - . AT1G78930 AT1G78930 AT1G78930 188
AT1G71695.1 three_prime_utr chr1:26966558-26966688 chr1 26966558 26966688 + . AT1G71695 AT1G71695 AT1G71695 131
AT1G58983.1 three_prime_utr chr1:21806020-21806278 chr1 21805932 21806278 - . AT1G58983 AT1G58983 AT1G58983 259
AT1G12980.1 three_prime_utr chr1:4430778-4430965 chr1 4430778 4430965 + . ESR1 AT1G12980 AT1G12980 188
AT1G45223.1 three_prime_utr chr1:17141620-17141771 chr1 17141620 17141771 + . AT1G45223 AT1G45223 AT1G45223 152
A

In [45]:
%%bash
cp GTF_three_prime_utr_final.txt ../PRUEBAS/
cd ../PRUEBAS/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt|head 

AT1G30814.1 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G30814.4 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10943007 10943138 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.2 three_prime_utr chr1:10942663-10943138 chr1 10942663 10942909 - . AT1G30814 AT1G30814 AT1G30814 379
AT1G30814.3 three_prime_utr chr1:10942648-10942825 chr1 10942648 10942825 - . AT1G30814 AT1G30814 AT1G30814 178
AT1G78930.1 three_prime_utr chr1:29677904-29678284 chr1 29677904 29678284 - . AT1G78930 AT1G78930 AT1G78930 381
AT1G71695.1 three_prime_utr chr1:26966558-26966688 chr1 26966558 26966688 + . AT1G71695 AT1G71695 AT1G71695 131
AT1G58983.1 three_prime_utr chr1:21805932-21806278 chr1 21805932 21806278 - . AT1G58983 AT1G58983 AT1G58983 347
AT1G12980.1 three_prime_utr chr1:4430778-4430965 chr1 4430778 4430965 + . ESR1 AT1G12980 AT1G12980 188
A

In [44]:
%%bash
cd ../PRUEBAS/
awk 'BEGIN{OFS="\t"} NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print arr[1], $3, arr[2], $1, $4, $5, $6, $7, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt |head

awk 'BEGIN{OFS="\t"} NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print arr[1], $3, arr[2], $1, $4, $5, $6, $7, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt > coords_3utr.dict

AT1G30814.1	three_prime_utr	chr1:10942648-10942825	chr1	10942648	.	10942825	-	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.4	three_prime_utr	chr1:10942648-10942825	chr1	10942648	.	10942825	-	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10943007	.	10943138	-	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10942663	.	10942909	-	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.3	three_prime_utr	chr1:10942648-10942825	chr1	10942648	.	10942825	-	AT1G30814	AT1G30814	AT1G30814	178
AT1G78930.1	three_prime_utr	chr1:29677904-29678284	chr1	29677904	.	29678284	-	AT1G78930	AT1G78930	AT1G78930	381
AT1G71695.1	three_prime_utr	chr1:26966558-26966688	chr1	26966558	26966688	.	+	AT1G71695	AT1G71695	AT1G71695	131
AT1G58983.1	three_prime_utr	chr1:21805932-21806278	chr1	21805932	.	21806278	-	AT1G58983	AT1G58983	AT1G58983	347
AT1G12980.1	three_prime_utr	chr1:4430778-4430965	chr1	4430778	4430965	.	+	ESR1	AT1G12980	AT1G12980	188
A

Claramente lo que hice arriba no esta bien, por lo que se procede a realizar lo siguiente

In [43]:
%%bash
cd ../PRUEBAS/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt|awk '{$1=$1};1' OFS='\t'|head

AT1G30814.1	three_prime_utr	chr1:10942648-10942825	chr1	10942648	10942825	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.4	three_prime_utr	chr1:10942648-10942825	chr1	10942648	10942825	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10943007	10943138	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10942663	10942909	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.3	three_prime_utr	chr1:10942648-10942825	chr1	10942648	10942825	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G78930.1	three_prime_utr	chr1:29677904-29678284	chr1	29677904	29678284	-	.	AT1G78930	AT1G78930	AT1G78930	381
AT1G71695.1	three_prime_utr	chr1:26966558-26966688	chr1	26966558	26966688	+	.	AT1G71695	AT1G71695	AT1G71695	131
AT1G58983.1	three_prime_utr	chr1:21805932-21806278	chr1	21805932	21806278	-	.	AT1G58983	AT1G58983	AT1G58983	347
AT1G12980.1	three_prime_utr	chr1:4430778-4430965	chr1	4430778	4430965	+	.	ESR1	AT1G12980	AT1G12980	188
A

Como eso funciona, lo unico que debemos de cambiar son las posiciones de START & End en el caso de las lecturas en sentido contrario

In [42]:
%%bash
cd ../PRUEBAS/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt|awk '{$1=$1};1' OFS='\t'|awk 'BEGIN{OFS="\t"} {if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print}'|head

AT1G30814.1	three_prime_utr	chr1:10942648-10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.4	three_prime_utr	chr1:10942648-10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10943138	10943007	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.2	three_prime_utr	chr1:10942663-10943138	chr1	10942909	10942663	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.3	three_prime_utr	chr1:10942648-10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G78930.1	three_prime_utr	chr1:29677904-29678284	chr1	29678284	29677904	-	.	AT1G78930	AT1G78930	AT1G78930	381
AT1G71695.1	three_prime_utr	chr1:26966558-26966688	chr1	26966558	26966688	+	.	AT1G71695	AT1G71695	AT1G71695	131
AT1G58983.1	three_prime_utr	chr1:21805932-21806278	chr1	21806278	21805932	-	.	AT1G58983	AT1G58983	AT1G58983	347
AT1G12980.1	three_prime_utr	chr1:4430778-4430965	chr1	4430778	4430965	+	.	ESR1	AT1G12980	AT1G12980	188
A

Como observamos, ahora funciona correctamente, lo unico que queda es cambiar el - entre las lecturas de la columna 3

In [41]:
%%bash

cd ../PRUEBAS/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' Araport11_3_utr_20220914.txt GTF_three_prime_utr_final.txt|awk '{$1=$1};1' OFS='\t'|awk 'BEGIN{OFS="\t"} {if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print}'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub("-", "..", $3); print}'|head

AT1G30814.1	three_prime_utr	chr1:10942648..10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.4	three_prime_utr	chr1:10942648..10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.2	three_prime_utr	chr1:10942663..10943138	chr1	10943138	10943007	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.2	three_prime_utr	chr1:10942663..10943138	chr1	10942909	10942663	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.3	three_prime_utr	chr1:10942648..10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G78930.1	three_prime_utr	chr1:29677904..29678284	chr1	29678284	29677904	-	.	AT1G78930	AT1G78930	AT1G78930	381
AT1G71695.1	three_prime_utr	chr1:26966558..26966688	chr1	26966558	26966688	+	.	AT1G71695	AT1G71695	AT1G71695	131
AT1G58983.1	three_prime_utr	chr1:21805932..21806278	chr1	21806278	21805932	-	.	AT1G58983	AT1G58983	AT1G58983	347
AT1G12980.1	three_prime_utr	chr1:4430778..4430965	chr1	4430778	4430965	+	.	ESR1	AT1G12980	AT1G12

PERFECTO, HASTA AQUI SE SOLUCIONA LA PARTE DE LA CREACION DE LOS ARCHIVOS

PERO, SE TIENE UN SERIO PROBLEMA, EL CUAL ES EL SIGUIENTE O SE TRABAJA CON ARAPORT11 O CON LOS DATOS DE TAIR10

POR LO QUE PRIMERO, VOY A TRABAJAR CON LOS DATOS DE TAIR10 de ensembl  
Aqui esta el GFT https://ftp.ebi.ac.uk/ensemblgenomes/pub/release-56/plants/gtf/arabidopsis_thaliana/  
Los fasta se encuentran aqui https://ftp.ebi.ac.uk/ensemblgenomes/pub/release-56/plants/fasta/arabidopsis_thaliana/  
->cdna  
->cds  
->dna  
->dna_index  
->ncrna  


PARA TRABAJAR CON LOS DATOS DE TAIR10 de arabidopsis.org  
Aqui estan los datos de TAIR10 https://www.arabidopsis.org/download/index-auto.jsp?dir=%2Fdownload_files%2FSequences%2FTAIR10_blastsets  
->3utr  
->5utr  
->cdna  
->cds  
->exons  
->intergenic  
->intron  
->seq  
->downstream  
->upstream  

PERO SI QUIERO TRABAJAR TODO CON TAIR11, EN DONDE ENCUENTRO LAS OTRAS SECUENCIAS???

Lo puedo descargar de esta pagina: https://www.arabidopsis.org/download/index-auto.jsp?dir=%2Fdownload_files%2FGenes%2FAraport11_genome_release

In [ ]:
%%bash
cd ../TAIR10/
wget --no-check-certificate "https://ftp.ebi.ac.uk/ensemblgenomes/pub/release-56/plants/gtf/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.56.gtf.gz"
wget --no-check-certificate "https://ftp.ebi.ac.uk/ensemblgenomes/pub/release-56/plants/fasta/arabidopsis_thaliana/ncrna/Arabidopsis_thaliana.TAIR10.ncrna.fa.gz"
wget --no-check-certificate "https://ftp.ebi.ac.uk/ensemblgenomes/pub/release-56/plants/fasta/arabidopsis_thaliana/dna_index/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.gz"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_3_utr_20101028"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_5_utr_20101028"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_exon_20101028"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_intergenic_20101028"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_intron_20101028"
wget --no-check-certificate "https://www.arabidopsis.org/download_files/Sequences/TAIR10_blastsets/TAIR10_seq_20101214"

In [40]:
%%bash
cd ../TAIR10/
ls

Arabidopsis_thaliana.TAIR10.56.gtf
Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
Arabidopsis_thaliana.TAIR10.ncrna.fa
gtf_tair10_3UTR.txt
TAIR10_3_utr_20101028.fa
TAIR10_5_utr_20101028.fa
TAIR10_exon_20101028.fa
TAIR10_intergenic_20101028.fa
TAIR10_intron_20101028.fa
TAIR10_seq_20101214.fa


In [ ]:
%%bash
cd ../TAIR10/
gzip -d *.gz
mv TAIR10_3_utr_20101028 TAIR10_3_utr_20101028.fa
mv TAIR10_5_utr_20101028 TAIR10_5_utr_20101028.fa
mv TAIR10_exon_20101028 TAIR10_exon_20101028.fa
mv TAIR10_intergenic_20101028 TAIR10_intergenic_20101028.fa
mv TAIR10_intron_20101028 TAIR10_intron_20101028.fa
mv TAIR10_seq_20101214 TAIR10_seq_20101214.fa


In [39]:
%%bash
cd ../TAIR10/
ls

Arabidopsis_thaliana.TAIR10.56.gtf
Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
Arabidopsis_thaliana.TAIR10.ncrna.fa
gtf_tair10_3UTR.txt
TAIR10_3_utr_20101028.fa
TAIR10_5_utr_20101028.fa
TAIR10_exon_20101028.fa
TAIR10_intergenic_20101028.fa
TAIR10_intron_20101028.fa
TAIR10_seq_20101214.fa


Lo primero que voy a ver es si ahora si se tienen la misma cantidad de lecturas para el 3'UTR en el GTF y en el fasta

In [79]:
%%bash
cd ../TAIR10/
grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|head 


1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.1"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-203"; transcript_source "araport11"; transcript_biotype "protein_coding"; tag "Ensembl_canonical";
1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.4"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-202"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10943007	10943138	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene_source "araport11"; gene_biotype "protein_coding"; transcript_name "AT1G30814-204"; transcript_source "araport11"; transcript_biotype "protein_coding";
1	araport11	three_prime_utr	10942663	10942909	.	-	.	gene_id "AT1G30814"; transcript_id "AT1G30814.2"; gene_name "AT1G30814"; gene

In [83]:
%%bash
cd ../TAIR10/
#grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'|head

grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'|awk '{$1 = "chr" $1; print}'|awk '{$1=$1};1' OFS='\t'|head

chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id	AT1G30814	transcript_id	AT1G30814.1	gene_name	AT1G30814	transcript_name	AT1G30814-203
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id	AT1G30814	transcript_id	AT1G30814.4	gene_name	AT1G30814	transcript_name	AT1G30814-202
chr1	araport11	three_prime_utr	10943007	10943138	.	-	.	gene_id	AT1G30814	transcript_id	AT1G30814.2	gene_name	AT1G30814	transcript_name	AT1G30814-204
chr1	araport11	three_prime_utr	10942663	10942909	.	-	.	gene_id	AT1G30814	transcript_id	AT1G30814.2	gene_name	AT1G30814	transcript_name	AT1G30814-204
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	gene_id	AT1G30814	transcript_id	AT1G30814.3	gene_name	AT1G30814	transcript_name	AT1G30814-201
chr1	araport11	three_prime_utr	29677904	29678284	.	-	.	gene_id	AT1G78930	transcript_id	AT1G78930.1	gene_name	AT1G78930	transcript_name	AT1G78930-201
chr1	araport11	three_prime_utr	26966558	26966688	.	+	.	gene_id	AT1G71695	transcript_id	AT1G71695.1	gene_na

Se ve bien pero aun se tiene que eliminar las palabras.

In [86]:
%%bash
cd ../TAIR10/
#grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'| sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|head
grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'|sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|awk '{$1 = "chr" $1; print}'|awk '{$1=$1};1' OFS='\t'|head

#grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'|sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'|awk '{$1 = "chr" $1; print}'|awk '{$1=$1};1' OFS='\t'|cut -f2|head

chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.1	AT1G30814	AT1G30814-203
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.4	AT1G30814	AT1G30814-202
chr1	araport11	three_prime_utr	10943007	10943138	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942663	10942909	.	-	.	AT1G30814	AT1G30814.2	AT1G30814	AT1G30814-204
chr1	araport11	three_prime_utr	10942648	10942825	.	-	.	AT1G30814	AT1G30814.3	AT1G30814	AT1G30814-201
chr1	araport11	three_prime_utr	29677904	29678284	.	-	.	AT1G78930	AT1G78930.1	AT1G78930	AT1G78930-201
chr1	araport11	three_prime_utr	26966558	26966688	.	+	.	AT1G71695	AT1G71695.1	AT1G71695	AT1G71695-201
chr1	araport11	three_prime_utr	21805932	21806278	.	-	.	AT1G58983	AT1G58983.1	AT1G58983	AT1G58983-201
chr1	araport11	three_prime_utr	4430778	4430965	.	+	.	AT1G12980	AT1G12980.1	ESR1	ESR1-201
chr1	araport11	three_prime_utr	17141620	17141771	.	+	.	AT1G45223	AT1G45223.1	AT1G45223	AT1G45223-201


Perfecto, ahora unicamente se debe de guardar el archivo.
## Este es el comando para el notebook final, para generar todos las anotaciones del GTF

In [87]:
%%bash
cd ../TAIR10/
grep -v '##\|#!' Arabidopsis_thaliana.TAIR10.56.gtf | grep 'three_prime_utr'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub(/"/, "", $9); split($9, arr, "; "); print $1, $2, $3, $4, $5, $6, $7, $8, arr[1], arr[2], arr[3], arr[6]}'| sed -e 's/gene_id //g' -e 's/transcript_id //g' -e 's/gene_name //g' -e 's/transcript_name //g'| awk '{$1 = "chr" $1; print}'| awk '{$1=$1};1' OFS='\t'> gtf_tair10_3UTR.txt

Muy bien, ahora con el archivo listo, se hace el recuento de cuantas anotaciones del 3UTR se tiene en el GTF de TAIR10

In [33]:
%%bash
cd ../TAIR10/
cut -f10  gtf_tair10_3UTR.txt|sort|uniq|wc -l

42322


Ahora toca realizar lo mismo, pero con el archivo del fasta

In [32]:
%%bash
cd ../TAIR10/
grep '>' TAIR10_3_utr_20101028.fa|head 

>AT1G51370.2 | [1135-1211] | chr1:19046749-19046825 FORWARD LENGTH=77
>AT1G50920.1 | [2433-2446] [2547-2792] | chr1:18872571-18872930 FORWARD LENGTH=260
>AT1G44020.1 | [1966-2545] | chr1:16716112-16716691 REVERSE LENGTH=580
>AT1G15970.1 | [2326-2550] | chr1:5486319-5486543 REVERSE LENGTH=225
>AT1G73440.1 | [821-1453] | chr1:27612183-27612815 FORWARD LENGTH=633
>AT1G75120.1 | [1782-1946] | chr1:28196857-28197021 REVERSE LENGTH=165
>AT1G17600.1 | [3640-3729] | chr1:6052936-6053025 REVERSE LENGTH=90
>AT1G51380.1 | [2087-2281] | chr1:19049968-19050162 FORWARD LENGTH=195
>AT1G77370.1 | [801-954] | chr1:29074643-29074796 FORWARD LENGTH=154
>AT1G10950.1 | [4415-4776] | chr1:3663623-3663984 FORWARD LENGTH=362


Basicamente me toca repetir lo que se hizo anteriromente:

In [31]:
%%bash
cd ../TAIR10/
grep '>' TAIR10_3_utr_20101028.fa | sed -e 's/|[^|]*|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t'|head

AT1G51370.2	chr1:19046749-19046825	FORWARD	77
AT1G50920.1	chr1:18872571-18872930	FORWARD	260
AT1G44020.1	chr1:16716112-16716691	REVERSE	580
AT1G15970.1	chr1:5486319-5486543	REVERSE	225
AT1G73440.1	chr1:27612183-27612815	FORWARD	633
AT1G75120.1	chr1:28196857-28197021	REVERSE	165
AT1G17600.1	chr1:6052936-6053025	REVERSE	90
AT1G51380.1	chr1:19049968-19050162	FORWARD	195
AT1G77370.1	chr1:29074643-29074796	FORWARD	154
AT1G10950.1	chr1:3663623-3663984	FORWARD	362


## Este es el comando para generar los encabezados del fasta

In [56]:
%%bash
cd ../TAIR10/
grep '>' TAIR10_3_utr_20101028.fa | sed -e 's/|[^|]*|//g' -e 's/>//g' -e 's/LENGTH=//g' | awk '{$1=$1};1' OFS='\t' > TAIR10_3_utr_20101028_head.txt

Este es el numero de anotaciones del fasta que son diferentes

In [58]:
%%bash
cd ../TAIR10/
cut -f1 TAIR10_3_utr_20101028_head.txt|sort|uniq|wc -l

28049


Eso es un problema, ya que en el GTF se tienen 42322 y en el fasta solo 28049.

Por lo que para tratar de solucionar este inconveniente, se optara por la siguiente estrategia:
Comparar los dos archivos y dejar unicamente los 28049 del fasta, pero tal vez esto no sea necesario, porque si son unicos, solo deberian de coincidir, los 28049 del fasta.

Primero debe de comprobar esto, haciendo la comparacion:

In [88]:
%%bash

cd ../TAIR10/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_head.txt gtf_tair10_3UTR.txt|awk '{$1=$1};1' OFS='\t'|awk 'BEGIN{OFS="\t"} {if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print}'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub("-", "..", $3); print}'|head

AT1G30814.1	three_prime_utr	chr1:10942648..10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G30814.2	three_prime_utr	chr1:10942663..10943138	chr1	10943138	10943007	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.2	three_prime_utr	chr1:10942663..10943138	chr1	10942909	10942663	-	.	AT1G30814	AT1G30814	AT1G30814	379
AT1G30814.3	three_prime_utr	chr1:10942648..10942825	chr1	10942825	10942648	-	.	AT1G30814	AT1G30814	AT1G30814	178
AT1G78930.1	three_prime_utr	chr1:29678097..29678284	chr1	29678284	29677904	-	.	AT1G78930	AT1G78930	AT1G78930	188
AT1G71695.1	three_prime_utr	chr1:26966558..26966688	chr1	26966558	26966688	+	.	AT1G71695	AT1G71695	AT1G71695	131
AT1G58983.1	three_prime_utr	chr1:21806020..21806278	chr1	21806278	21805932	-	.	AT1G58983	AT1G58983	AT1G58983	259
AT1G12980.1	three_prime_utr	chr1:4430778..4430965	chr1	4430778	4430965	+	.	ESR1	AT1G12980	AT1G12980	188
AT1G45223.1	three_prime_utr	chr1:17141620..17141771	chr1	17141620	17141771	+	.	AT1G45223	AT1G45223	AT1G45

In [89]:
%%bash

cd ../TAIR10/
awk 'NR==FNR{a[$1]=$0; next} $10 in a{split(a[$10], arr, " "); print arr[1], $3, arr[2], $1, $4, $5, $7, $6, $11, $9, $9, arr[4]}' TAIR10_3_utr_20101028_head.txt gtf_tair10_3UTR.txt|awk '{$1=$1};1' OFS='\t'|awk 'BEGIN{OFS="\t"} {if ($7 == "-") {temp = $5; $5 = $6; $6 = temp} else if ($7 == "+") {$5 = $5; $6 = $6} print}'|awk -F'\t' 'BEGIN{OFS="\t"} {gsub("-", "..", $3); print}'>3UTR.coords.dict

El comando funciona, ahora toca contar el numero de contactos totales y unicos:

In [90]:
%%bash
cd ../TAIR10/
wc -l 3UTR.coords.dict
cut -f1 3UTR.coords.dict|sort|uniq|wc -l

30316 3UTR.coords.dict
27719


Vemos que los uniq son diferentes, ahora con el siguiente script se va a imprimir solo las coincidencias de los archivos con el id del transcrito

In [91]:
import os

directorio = "../TAIR10/"
os.chdir(directorio)

def comparar_archivos(archivo1, archivo2, archivo_salida):
    # Leer el contenido del archivo 1
    with open(archivo1, 'r') as file1:
        columnas_archivo1 = [line.split('\t')[0] for line in file1]  # Obtener los valores de la columna 1

    # Leer el contenido del archivo 2
    with open(archivo2, 'r') as file2:
        columnas_archivo2 = [line.split('\t')[9] for line in file2]  # Obtener los valores de la columna 10

    # Comparar y escribir las coincidencias en el archivo de salida
    coincidencias = set(columnas_archivo1) & set(columnas_archivo2)
    with open(archivo_salida, 'w') as file_salida:
        for coincidencia in coincidencias:
            print(coincidencia, file=file_salida)

# Ejemplo de uso
archivo1 = 'TAIR10_3_utr_20101028_head.txt'
archivo2 = 'gtf_tair10_3UTR.txt'
archivo_salida = 'resultados.txt'
comparar_archivos(archivo1, archivo2, archivo_salida)





In [92]:
%%bash
cd ../TAIR10/

wc -l resultados.txt

sort resultados.txt|uniq|wc -l

27719 resultados.txt
27719


Como el resultado es el mismo que el que se obtiene haciendo uniq al comando de awk, al menos esto tiene consistencia.

Ordeno el archivo y veo que le pasa a los repetidos

In [95]:
%%bash
cd ../TAIR10/
sort -k1,1 -r 3UTR.coords.dict|head -n 20
echo --------------------------------------------------------------------------------------------------------------------------------------------------
sort -k1,1 -r 3UTR.coords.dict|tail -n 20


AT5G67630.1	three_prime_utr	chr5:26967381..26967534	chr5	26967534	26967378	-	.	AT5G67630	AT5G67630	AT5G67630	154
AT5G67620.1	three_prime_utr	chr5:26964770..26964890	chr5	26964890	26964551	-	.	AT5G67620	AT5G67620	AT5G67620	121
AT5G67610.2	three_prime_utr	chr5:26960931..26961223	chr5	26961223	26960931	-	.	AT5G67610	AT5G67610	AT5G67610	293
AT5G67610.1	three_prime_utr	chr5:26960992..26961223	chr5	26961223	26960904	-	.	AT5G67610	AT5G67610	AT5G67610	232
AT5G67600.1	three_prime_utr	chr5:26959569..26959753	chr5	26959753	26959533	-	.	WIH1	AT5G67600	AT5G67600	185
AT5G67590.1	three_prime_utr	chr5:26959357..26959557	chr5	26959357	26959557	+	.	FRO1	AT5G67590	AT5G67590	201
AT5G67580.2	three_prime_utr	chr5:26955135..26955842	chr5	26955842	26955568	-	.	TRB2	AT5G67580	AT5G67580	708
AT5G67580.1	three_prime_utr	chr5:26955135..26955842	chr5	26955842	26955568	-	.	TRB2	AT5G67580	AT5G67580	708
AT5G67570.1	three_prime_utr	chr5:26955481..26955543	chr5	26955481	26955543	+	.	DG1	AT5G67570	AT5G67570	63
AT5G67560.